# 📁 01. 데이터 전처리 및 통합

In [1]:
import pandas as pd

## 데이터 세트 전처리

### 🌟 공통 전처리
- **기간** : 2021년 데이터 기준
- **지역** : 서울특별시, 경기도, 인천광역시
- **지역 통합 처리**
    - 경기도, 인천광역시 데이터: '구' → '시' 단위로 통합(평균)
    - '중구' 중복 구분: 서울중구, 인천중구로 명시
- **고유값 설정** : 지역구명을 고유 인덱스로 설정하여 데이터 병합

- - -

### 💤 수면장애 데이터 전처리
- 출처: 국민건강보험공단_수면장애 관련 주상병별 진료정보
- 사용 조건:
    - **2021**년도
    - 주상병명 = **'비기질성 수면장애'**
- 사용 변수: `시군구`, `진료인원(명)`

In [ ]:
# 데이터 불러오기
data = pd.read_csv('data/수면장애 관련 주상병별 진료정보_2020.csv')
data.head()

,주상병명,진료년도,시도,시군구,진료인원(명),시군구2
0,비기질성 수면장애,2021,서울특별시,종로구,1202,종로구
1,비기질성 수면장애,2021,서울특별시,중구,1280,중구
2,비기질성 수면장애,2021,서울특별시,용산구,1955,용산구
3,비기질성 수면장애,2021,서울특별시,성동구,2256,성동구
4,비기질성 수면장애,2021,서울특별시,광진구,2351,광진구


In [16]:
# 진료년도 컬럼 삭제(모두 2021년으로 동일)
data.drop(columns='진료년도', inplace=True)
data.columns

Index(['주상병명', '시도', '시군구', '진료인원(명)', '시군구2'], dtype='object')

In [17]:
# 중구 중복 처리
data.loc[(data['시도']=='서울특별시')&(data['시군구']=='중구'), '시군구'] = '서울중구'
data.loc[(data['시도']=='인천광역시')&(data['시군구']=='중구'), '시군구'] = '인천중구'

# 시군구 대분류(시군구2) 중구 중복 처리
data.loc[(data['시도']=='서울특별시')&(data['시군구2']=='중구'), '시군구2'] = '서울중구'
data.loc[(data['시도']=='인천광역시')&(data['시군구2']=='중구'), '시군구2'] = '인천중구'

In [18]:
# 시군구 고유값인지 확인
if (data['시군구'].value_counts() > 1).any():
    print('시군구 고유값이 아닙니다.')
else:
    print('시군구 고유값입니다.')

시군구 고유값입니다.


In [19]:
# 중복된 시군구명 행을 첫 행만 남기고 나머지 삭제
data = data.drop_duplicates(subset=['시군구2'], keep='first')

# 빈도 컬럼을 중복된 시군구명의 평균값으로 대체
data['진료인원(명)'] = data.groupby('시군구2')['진료인원(명)'].transform('mean')

# 중복된 시군구명 행을 다시 삭제하여 평균값이 반영된 첫 행만 남기기
data = data.drop_duplicates(subset=['시군구2'], keep='first')

In [20]:
if (data['시군구2'].value_counts() > 1).any():
    print('중복된 시군구2 값:', data['시군구2'].value_counts()[data['시군구2'].value_counts() > 1].index.tolist())
else:
    print('중복없음')

중복없음


### 🍔 야식(배달) 데이터 전처리   

1) 야식 빈도 데이터 전처리
    - `세대구성대분류명`을 시군구별로 평균 집계하여 컬럼화
    - '다섯가구 미만' → '5'로 대체하여 수치형으로 변환

In [ ]:
# 주거 특성별 배달 데이터 불러오기
data2 = pd.read_csv('data/delivery.csv', index_col=0)
data2.head()

,년,광역시도명,시군구명,세대구성대분류명,빈도,시군구2
0,2020,서울특별시,종로구,1세대 가구,8752.0,종로구
1,2020,서울특별시,종로구,1세대 가구,3162.0,종로구
2,2020,서울특별시,종로구,1세대 가구,2628.0,종로구
3,2020,서울특별시,종로구,1세대 가구,836.0,종로구
4,2020,서울특별시,종로구,1세대 가구,1406.0,종로구


In [23]:
# 년도 컬럼 삭제
data2.drop(columns=['년'], inplace=True)

# 광역시도명 -> 시도/ 시군구명 -> 시군구로 컬럼명 변경
data2.rename(columns={'광역시도명':'시도'}, inplace=True)
data2.rename(columns={'시군구명':'시군구'}, inplace=True)

In [24]:
# 중구 중복 처리
data2.loc[(data2['시도']=='서울특별시')&(data2['시군구']=='중구'), '시군구'] = '서울중구'
data2.loc[(data2['시도']=='인천광역시')&(data2['시군구']=='중구'), '시군구'] = '인천중구'

# 중구 중복 처리
data2.loc[(data2['시도']=='서울특별시')&(data2['시군구2']=='중구'), '시군구2'] = '서울중구'
data2.loc[(data2['시도']=='인천광역시')&(data2['시군구2']=='중구'), '시군구2'] = '인천중구'

In [25]:
# 세대구성대분류명 확인
data2['세대구성대분류명'].unique()

array(['1세대 가구', '2세대 가구', '3세대 가구', '4세대 이상 가구', '1인 가구', '비친족 가구'],
      dtype=object)

In [26]:
# 시군구별 세대구성대분류명의 빈도 평균 계산 (컬럼화)
data_pivot = data2.pivot_table(index='시군구', columns='세대구성대분류명', values='빈도', aggfunc='mean')

# 인덱스 초기화
data_pivot = data_pivot.reset_index()

# 세대구성대분류 대체
data_final = data2.drop(columns=['세대구성대분류명', '빈도']).drop_duplicates(subset=['시군구'])

# 기존 데이터와 병합
data2 = pd.merge(data_final, data_pivot, on='시군구', how='left')
data2.head()

,시도,시군구,시군구2,1세대 가구,1인 가구,2세대 가구,3세대 가구,4세대 이상 가구,비친족 가구
0,서울특별시,종로구,종로구,348.633333,5198.666667,507.567407,119.103333,6.186667,380.753333
1,서울특별시,서울중구,서울중구,314.186667,4565.666667,427.126667,94.586667,5.173333,381.146667
2,서울특별시,용산구,용산구,553.745556,7378.266667,792.157778,158.685000,6.453333,709.946667
3,서울특별시,성동구,성동구,712.411111,8519.066667,1135.387407,228.603333,7.206667,541.346667
4,서울특별시,광진구,광진구,772.342222,12462.266667,1296.900000,247.738333,6.533333,688.546667


In [27]:
# 중복된 시군구명 행을 첫 행만 남기고 나머지 삭제
data2 = data2.drop_duplicates(subset=['시군구2'], keep='first')

# 빈도 컬럼을 중복된 시군구명의 평균값으로 대체
columns_to_transform = ['1세대 가구', '2세대 가구', '3세대 가구', '4세대 이상 가구', '1인 가구', '비친족 가구']
data2[columns_to_transform] = data2.groupby('시군구2')[columns_to_transform].transform('mean')

# 중복된 시군구명 행을 다시 삭제하여 평균값이 반영된 첫 행만 남기기
data2 = data2.drop_duplicates(subset=['시군구2'], keep='first')

In [28]:
# 시군구 고유값인지 확인
if (data2['시군구2'].value_counts() > 1).any():
    print('시군구 고유값이 아닙니다.')
else:
    print('시군구 고유값입니다.')

시군구 고유값입니다.


2) 야식 평균주문금액 데이터 전처리
    - `평균주문금액`을 시군구 기준으로 평균 집계
    - 날짜, 시간대 관련 컬럼 제거

In [ ]:
# 주문 가격 데이터 불러오기
data3 = pd.read_csv('data/order_price.csv', index_col=0)
data3.head()

,날짜,시간대별 시간,목적지 광역시도명,목적지 시군구명,평균주문금액,시군구2
0,2021,0,경기도,동두천시,72667,동두천시
1,2021,0,경기도,부천시,34877,부천시
2,2021,0,경기도,안산시 상록구,24420,안산시
3,2021,0,경기도,의정부시,31600,의정부시
6,2021,0,서울특별시,구로구,34345,구로구


In [36]:
# 날짜, 시군구2 컬럼 삭제
data3.drop(columns=['날짜'], inplace=True)

# 목적지 광역시도명 -> 시도, 목적지 시군구명 -> 시군구로 컬럼명 변경
data3.rename(columns={'목적지 광역시도명':'시도'}, inplace=True)
data3.rename(columns={'목적지 시군구명':'시군구'}, inplace=True)

In [37]:
# 중구 중복 처리
data3.loc[(data3['시도']=='서울특별시')&(data3['시군구']=='중구'), '시군구'] = '서울중구'
data3.loc[(data3['시도']=='인천광역시')&(data3['시군구']=='중구'), '시군구'] = '인천중구'

# 중구 중복 처리
data3.loc[(data3['시도']=='서울특별시')&(data3['시군구2']=='중구'), '시군구2'] = '서울중구'
data3.loc[(data3['시도']=='인천광역시')&(data3['시군구2']=='중구'), '시군구2'] = '인천중구'

In [38]:
# 야간평균주문금액

# 시간대별 시간 컬럼 삭제
data3.drop(columns=['시간대별 시간'], inplace=True)

# 시군구별 평균주문금액 계산
df_g = data3.groupby('시군구', as_index=False)['평균주문금액'].mean()

# 기존 데이터에서 '평균주문금액' 컬럼 대체
data3 = pd.merge(
    data3.drop(columns=['평균주문금액']).drop_duplicates(subset=['시군구']),
    df_g, on='시군구', how='left'
    )
data3.head()

,시도,시군구,시군구2,평균주문금액
0,경기도,동두천시,동두천시,26882.143657
1,경기도,부천시,부천시,28329.552441
2,경기도,안산시 상록구,안산시,27154.460281
3,경기도,의정부시,의정부시,28458.182341
4,서울특별시,구로구,구로구,28139.356929


In [39]:
# 중복된 시군구명 행을 첫 행만 남기고 나머지 삭제
data3 = data3.drop_duplicates(subset=['시군구2'], keep='first')

# 빈도 컬럼을 중복된 시군구명의 평균값으로 대체
data3['평균주문금액'] = data3.groupby('시군구2')['평균주문금액'].transform('mean')

# 중복된 시군구명 행을 다시 삭제하여 평균값이 반영된 첫 행만 남기기
data3 = data3.drop_duplicates(subset=['시군구2'], keep='first')

In [40]:
# 시군구 고유값인지 확인
if (data3['시군구2'].value_counts() > 1).any():
    print('시군구 고유값이 아닙니다.')
else:
    print('시군구 고유값입니다.')

시군구 고유값입니다.


### 🏙️ 유흥업소 데이터 전처리

- 유흥업소 개수 카운트 → 시군구별로 밀집도 수치화
- 결측치 구는 삭제
- 서울 + 인천/경기 : 구 단위 통일 후 병합

1) 서울

In [ ]:
# 데이터 불러오기
alcohol_seoul = pd.read_csv('data/서울_유흥업소_구별분류.csv')
alcohol_seoul.head()

,상세영업상태코드,지번주소,사업장명,데이터갱신일자,구
0,1,서울특별시 강남구 역삼동 678-24 늘봄빌딩,골드코스트,2023-12-07 00:07:00.0,강남구
1,1,서울특별시 송파구 가락동 98-7,골드1,2023-12-03 22:06:00.0,송파구
2,2,서울특별시 마포구 합정동 382-22번지 지하1층,??,2018-08-31 23:59:59.0,마포구
3,1,서울특별시 영등포구 신길동 449-22 신성빌딩,린스,2022-10-31 22:02:00.0,영등포구
4,1,서울특별시 서초구 잠원동 12-21,만남,2022-12-05 22:09:00.0,서초구


In [42]:
# 중구 -> 서울중구 변경
def change(name):
    if name == '중구':
        return '서울중구'
    return name

alcohol_seoul['구'] = alcohol_seoul['구'].apply(change)

In [43]:
# 결측치 제거
alcohol_seoul = alcohol_seoul.dropna(subset='구')

In [44]:
# 사업장 수 카운트
alcohol_seoul2 = alcohol_seoul.groupby(by='구', as_index=False).count()

In [45]:
# 사업장 수 컬럼만 남기기
alcohol_seoul2 = alcohol_seoul2.drop(['상세영업상태코드','지번주소','데이터갱신일자'], axis=1)
alcohol_seoul2.rename(columns={'사업장명':'유흥업소 개수'}, inplace=True)
alcohol_seoul2.head()

,구,유흥업소 개수
0,강남구,658
1,강동구,288
2,강북구,154
3,강서구,232
4,관악구,423


2) 인천&경기

In [ ]:
# 데이터 불러오기
alcohol_gg = pd.read_csv('data/인천경기_유흥업소.csv')

alcohol_gg.head()

,상세영업상태명,도로명전체주소,사업장명,구,시도,시군구2
0,영업,인천광역시 옹진군 백령면 백령로 298-1,장미노래클럽,NaN,인천광역시,옹진군
1,영업,인천광역시 옹진군 영흥면 영흥남로 307,빠샤 노래주점,NaN,인천광역시,옹진군
2,영업,인천광역시 옹진군 영흥면 영흥북로35번길 21,뉴 차차차 노래클럽,NaN,인천광역시,옹진군
3,영업,인천광역시 옹진군 백령면 백령로 282-1,퍼펙트,NaN,인천광역시,옹진군
4,영업,"인천광역시 옹진군 백령면 백령로 287, 1층",텔레파시,NaN,인천광역시,옹진군


In [47]:
# 중구 -> 인천중구 변경
def change2(name):
    if name == '중구':
        return '인천중구'
    return name

alcohol_gg['구'] = alcohol_gg['시군구2'].apply(change2)

In [48]:
# 사업장 수 카운트
alcohol_gg2 = alcohol_gg.groupby(by='구', as_index=False).count()

In [49]:
# 사업장 수 컬럼만 남기기
alcohol_gg2 = alcohol_gg2.drop(['상세영업상태명','도로명전체주소','시도','시군구2'], axis=1)
alcohol_gg2.rename(columns={'사업장명':'유흥업소 개수'}, inplace=True)
alcohol_gg2.columns

Index(['구', '유흥업소 개수'], dtype='object')

In [50]:
alcohol_gg2[alcohol_gg2['구'] =='안양시']

,구,유흥업소 개수
21,안양시,338


In [51]:
# 유흥업소 데이터 통합
alcohol = pd.concat([alcohol_seoul2,alcohol_gg2], ignore_index=True)
alcohol.rename(columns={'구':'시군구2'}, inplace=True)
alcohol.head()

,시군구2,유흥업소 개수
0,강남구,658
1,강동구,288
2,강북구,154
3,강서구,232
4,관악구,423


### 🏃 활동량 데이터 전처리

- 기준연월: 2020년 9월 (2021 데이터 없음)
- 사용 변수: `시군구별 시설 수`,`인구 수` → 인당 시설 수 계산

In [ ]:
# 데이터 불러오기
data4 = pd.read_csv('data/activity.csv', index_col=0)
data4.head()

,기준연월,시도명,시군구명,시군구별시설수,시군구별인구수,인당시설수,시군구2
0,202008,서울특별시,종로구,81,149962,0.00054,종로구
1,202008,서울특별시,중구,26,125990,0.00021,중구
2,202008,서울특별시,용산구,28,229362,0.00012,용산구
3,202008,서울특별시,성동구,28,296183,0.00009,성동구
4,202008,서울특별시,광진구,25,348647,0.00007,광진구


In [53]:
# 9월 데이터만 추출
act09 = data4[data4['기준연월'] == 202009]
act09

,기준연월,시도명,시군구명,시군구별시설수,시군구별인구수,인당시설수,시군구2
229,202009,서울특별시,종로구,81,149952,0.00054,종로구
230,202009,서울특별시,중구,26,125800,0.00021,중구
231,202009,서울특별시,용산구,28,229786,0.00012,용산구
232,202009,서울특별시,성동구,28,295591,0.00009,성동구
233,202009,서울특별시,광진구,25,348064,0.00007,광진구
...,...,...,...,...,...,...,...
330,202009,경기도,포천시,102,147803,0.00069,포천시
331,202009,경기도,여주시,69,111512,0.00062,여주시
332,202009,경기도,연천군,78,43590,0.00179,연천군
333,202009,경기도,가평군,76,62551,0.00122,가평군


In [ ]:
# 시군구, 시도 컬럼명 변경
act09.rename(columns={'시군구명':'시군구'}, inplace=True)
act09.rename(columns={'시도명':'시도'}, inplace=True)

In [ ]:
# 중복된 시군구명 행을 첫 행만 남기고 나머지 삭제
act09 = act09.drop_duplicates(subset=['시군구'], keep='first')

# 빈도 컬럼을 중복된 시군구명의 총합 대체
list = ['시군구별시설수','시군구별인구수','인당시설수']
act09[list] = act09.groupby('시군구')[list].transform('sum')

# 중복된 시군구명 행을 다시 삭제하여 총합이 반영된 첫 행만 남기기
act09 = act09.drop_duplicates(subset=['시군구'], keep='first')

In [56]:
# 중구 중복 처리
act09.loc[(act09['시도']=='서울특별시')&(act09['시군구']=='중구'), '시군구'] = '서울중구'
act09.loc[(act09['시도']=='인천광역시')&(act09['시군구']=='중구'), '시군구'] = '인천중구'

In [57]:
# 시군구 고유값인지 확인
if (act09['시군구2'].value_counts() > 1).any():
    print('시군구 고유값이 아닙니다.')
else:
    print('시군구 고유값입니다.')

시군구 고유값입니다.


In [58]:
act09.drop(columns=['시군구'], inplace=True)

### 🚨 범죄율 데이터 전처리

- 사용 변수: `총신고수`
- 불필요한 신고 항목 컬럼 제거
- 기준 년도: 2020 

In [ ]:
data5 = pd.read_csv('data/범죄 빈발지 데이터 2020.csv',encoding='cp949')
data5.head()

,신고년도,시도,시군구,총신고수,가정폭력신고수,데이트폭력신고수,성폭력신고수,몰래카메라신고수,살인신고수,강도신고수,절도신고수,폭력신고수
0,2020,경기도,가평군,1141,184,59,28,0,0,2,273,595
1,2020,경기도,고양시,26557,6148,845,648,9,12,10,5746,13175
2,2020,경기도,과천시,603,165,7,24,0,0,0,183,224
3,2020,경기도,광명시,26100,6314,1080,666,0,2,20,6100,11918
4,2020,경기도,광주시,6238,2260,253,111,1,0,0,1283,2331


In [61]:
# 중구 중복 처리
data5.loc[(data5['시도']=='서울특별시')&(data5['시군구']=='중구'), '시군구'] = '서울중구'
data5.loc[(data5['시도']=='인천광역시')&(data5['시군구']=='중구'), '시군구'] = '인천중구'

In [62]:
# 총신고수 제외 컬럼 삭제
data5 = data5[['시군구', '총신고수']]
data5.rename(columns={'시군구': '시군구2'}, inplace=True)

### 🚗 교통량 데이터 전처리

- 22시 ~ 06시 시간대 필터링(야간 통행량만 계산)
- 유입 교통량 평균 → `유입야간교통량` 컬럼 생성

In [ ]:
# 데이터 불러오기
data6 = pd.read_csv('data/야간교통량 평균.csv')
data6.head()

,도로명,구 이름,유입야간교통량평균
0,A-01 성산로(금화터널,서대문구,746.545455
1,A-02 사직로(사직터널,종로구,867.636364
2,A-03 자하문로(자하문터널,종로구,347.454545
3,A-04 대사관로(삼청터널,성북구,80.545455
4,A-05 율곡로(안국역,종로구,695.909091


In [68]:
data6.loc[data6['구 이름']=='중구','구 이름'] = '서울중구'

In [69]:
data6 = data6.groupby(by='구 이름',as_index=False)['유입야간교통량평균'].mean()

In [70]:
# 구이름 -> 시군구 컬럼명 변경
data6.rename(columns={'구 이름':'시군구2'}, inplace=True)

### 💰 공시지가 데이터 전처리

- 시군구 기준으로 평균 공시지가 계산
- 주거지역 필터링

In [ ]:
data7 = pd.read_csv('data/공시지가_수정.csv', encoding='cp949')
data7.head()

,시군구명,공시지가,공시지가2
0,가평군,1.425912e+05,0.008486
1,강남구,1.680318e+07,1.000000
2,강동구,5.667941e+06,0.337314
3,강북구,3.519811e+06,0.209473
4,강서구,4.261157e+06,0.253592


In [73]:
# 시군구명 -> 시군구로 컬럼명 변경
data7.rename(columns={'시군구명':'시군구2'}, inplace=True)

## 데이터 병합

- 기준: 수면장애 데이터의 시군구
- 방식: left merge
- 병합 후 결측치 확인 및 간단한 처리

In [74]:
data3.drop(columns=['시도'], inplace=True)
data2.drop(columns=['시도'], inplace=True)
act09.drop(columns=['시도'], inplace=True)

In [75]:
# 데이터 병합
data_merge = pd.merge(data, data2, on='시군구2', how='left').merge(data3, on='시군구2', how='left')
data_merge = pd.merge(data_merge, alcohol, on='시군구2', how='left')
data_merge = pd.merge(data_merge, act09, on='시군구2', how='left')
data_merge = pd.merge(data_merge, data5, on='시군구2', how='left')
data_merge = pd.merge(data_merge, data6, on='시군구2', how='left')
data_merge = pd.merge(data_merge, data7, on='시군구2', how='left')

# 병합된 데이터 확인
data_merge.head()

,주상병명,시도,시군구_x,진료인원(명),시군구2,시군구_y,1세대 가구,1인 가구,2세대 가구,3세대 가구,...,평균주문금액,유흥업소 개수,기준연월,시군구별시설수,시군구별인구수,인당시설수,총신고수,유입야간교통량평균,공시지가,공시지가2
0,비기질성 수면장애,서울특별시,종로구,1202.0,종로구,종로구,348.633333,5198.666667,507.567407,119.103333,...,24024.285714,439.0,202009.0,81.0,149952.0,0.00054,6834.0,524.102273,9.045009e+06,0.538291
1,비기질성 수면장애,서울특별시,서울중구,1280.0,서울중구,서울중구,314.186667,4565.666667,427.126667,94.586667,...,27154.801724,656.0,NaN,NaN,NaN,NaN,10584.0,441.578512,1.399032e+07,0.832600
2,비기질성 수면장애,서울특별시,용산구,1955.0,용산구,용산구,553.745556,7378.266667,792.157778,158.685000,...,26419.737778,88.0,202009.0,28.0,229786.0,0.00012,6923.0,901.952273,9.666904e+06,0.575302
3,비기질성 수면장애,서울특별시,성동구,2256.0,성동구,성동구,712.411111,8519.066667,1135.387407,228.603333,...,19055.000000,33.0,202009.0,28.0,295591.0,0.00009,6164.0,838.563636,6.484237e+06,0.385893
4,비기질성 수면장애,서울특별시,광진구,2351.0,광진구,광진구,772.342222,12462.266667,1296.900000,247.738333,...,NaN,26.0,202009.0,25.0,348064.0,0.00007,18288.0,868.400000,5.778178e+06,0.343874


In [76]:
# 중복 시군구 행 삭제
data_merge.drop(columns=['시군구_x','시군구_y','시군구','기준연월'], inplace=True)

In [ ]:
# 데이터 저장
data_merge.to_csv('data/통합데이터.csv', index=False)